# Lista de exercícios 7 - MNUM-7077

## Exercício 1 (10 pontos)

#### Antonio C. da Silva Júnior

Bibliotecas e funções úteis:

In [1]:
import itertools, copy, time
import numpy as np
import pyomo.environ as pyo
import pandas as pd
from statistics import *

In [2]:
# Função para criação dos conjuntos que irão originar as restrições de sub-rota de Dantzig
def obter_conjuntos_dantzig(cij):
    n = len(cij)

    # Combinações:
    c = []
    for i in range(n):
        c.append([0,1])

    comb = np.array(list(itertools.product(*c)))
    comb = np.flip(comb, axis=1)
    comb = comb[(np.sum(comb, axis=1) > 1) & (np.sum(comb, axis=1) < (n-1))] # remove todos os vazis, cheios e redundantes

    # Definição dos conjuntos Q e não Q:
    Q = []
    nQ = []
    for i in range(len(comb)):
        q = []
        n_q = []
        for j in range(len(comb[i])):
            if comb[i,j] == 1:
                q.append(j+1)
            else:
                n_q.append(j+1)
        Q.append(q)
        nQ.append(n_q)

    return Q, nQ

In [3]:
# Função para declarar um modelo de designação:

def declarar_modelo_designacao(cij):
    
    n = len(cij)

    # Declaração do modelo:
    modelo = pyo.ConcreteModel()

    # Índices dos pontos:
    modelo.M = pyo.RangeSet(n)
    modelo.N = pyo.RangeSet(n)

    # Matriz de custos:
    modelo.c = pyo.Param(modelo.N, modelo.M, initialize=lambda modelo_D, i, j: cij[i-1][j-1])

    # Variáveis de decisão:
    modelo.x = pyo.Var(modelo.N, modelo.M, within=pyo.Binary)

    # Função objetivo:
    def f_objetivo(modelo):
        return sum(modelo.x[i,j] * modelo.c[i,j] for i in modelo.N for j in modelo.M)

    modelo.objetivo = pyo.Objective(rule=f_objetivo, sense=pyo.minimize)

    # Restrições

    # Cada ponto só recebe de uma única origem:
    def f_restr1(modelo, M):
        return sum(modelo.x[i,M] for i in modelo.N if i!= M) == 1

    modelo.restr1 = pyo.Constraint(modelo.M, rule=f_restr1)

    # Cada ponto só envia para um destino:
    def f_restr2(modelo, N):
        return sum(modelo.x[N,j] for j in modelo.M if j != N) == 1

    modelo.restr2 = pyo.Constraint(modelo.N, rule=f_restr2)
    
    return modelo

In [4]:
# Função que resolve o modelo n vezes e devolve o resultado e um vetor com o tempo de cada execução:

def resolver_modelo(modelo, n_exec=50):
    t = []
    for i in range(n_exec):
        solver = pyo.SolverFactory('glpk')
        result = solver.solve(modelo)
        t.append(result.Solver.Time)
    return t, result

In [5]:
# Função para visualizar os arcos do trajeto:

def obter_trajeto(modelo, cij):
    n = len(cij)
    
    l = list(modelo.x.keys())
    trajeto = []
    for i in l:
        if modelo.x[i]() is not None and modelo.x[i]() != 0:
            trajeto.append(i)
            
            
    # Ordena o trajeto:
    trajeto_ordenado = [trajeto[0]]
    while len(trajeto_ordenado) < n:
        for i in range(n):
            if i+1 == n:
                for proximo_arco in trajeto:
                    if proximo_arco not in trajeto_ordenado:
                        trajeto_ordenado.append(proximo_arco)
                        break
            else:
                ultimo_arco = trajeto_ordenado[len(trajeto_ordenado)-1]
                for arco_atual in trajeto:
                    if arco_atual not in trajeto_ordenado:
                        if arco_atual[0] == ultimo_arco[1]:
                            trajeto_ordenado.append(arco_atual)
                            break
    
    print(trajeto_ordenado)
    
    return trajeto_ordenado

In [6]:
# Cálcula a média e o desvio padrão do tempo de execução:

def obter_estatisticas(tempo):
    media = round(mean(tempo), 4)
    dp = round(stdev(tempo), 4)
    msg = " Tempo médio: {media} \n Desvio padrão: {dp}".format(media=media, dp=dp)
    print(msg)

In [7]:
# Função para identificar subrotas:

def obter_subrotas(trajeto):
        
    # Separa as subrotas, caso haja:
    subrota = []
    subrotas = []
    for arco in trajeto:
        if len(subrota) == 0:
            subrota.append(arco)
        else:
            if arco[0] == subrota[len(subrota)-1][1]:
                subrota.append(arco)
            else:
                subrotas.append(subrota)
                subrota = []
                subrota.append(arco)
    subrotas.append(subrota)
    
    if(len(subrotas) > 1):
        msg = "Há {n} subrotas!".format(n=len(subrotas))
    else:
        msg = "Não há subrotas!"
        
    print(msg)
    print(subrotas)
        
    return subrotas

In [8]:
# Função para calcular o custo das subrotas

def obter_custo_subrotas(subrotas):
    dict_subrotas = {}
    for i in range(len(subrotas)):
        sub = subrotas[i]
        custo = 0
        for arco in sub:
            custo += cij[arco[0]-1][arco[1]-1]
        dict_subrotas[i] = {'subrota': sub,
                            'custo': round(custo,2),}

    print(dict_subrotas)
    
    return dict_subrotas

In [9]:
 # Calcula o custo do trajeto (quando em formato de lista):
    
def obter_custo_trajeto(trajeto, cij):
    custos = []
    for i in range(len(trajeto)):
        if i == 0:
            custos.append(0)
        else:
            no_anterior = trajeto[i-1]
            no_atual = trajeto[i]
            custo = cij[no_anterior][no_atual]
            custos.append(custo)
    return sum(custos)

In [10]:
# Função para resolver pela heurística de inserção do mais próximo ou mais distante

def resolver_heuristica_insercao(cij, no_inicial=1, mais_distante=False):
    
    no_inicial -= no_inicial
    n = len(cij) # Número de pontos:
    trajeto = [no_inicial] # Inclui o nó inicial no trajeto:

    # Identifica o nó mais próximo (ou mais distante) dos nós do trajeto:
    def escolher_no(trajeto, cij, no_inicial, mais_distante):
        no_escolhido = None
        no_ref = None
        if mais_distante:
            custo_final = -1*float('inf')
        else:
            custo_final = float('inf')

        for no in trajeto:
            for j in range(n):
                custo = cij[no][j]
                if j!= no and j not in trajeto:
                    if mais_distante:
                        if custo > custo_final:
                            custo_final = custo
                            no_escolhido = j
                            no_ref = no
                    else:
                        if custo < custo_final:
                            custo_final = custo
                            no_escolhido = j
                            no_ref = no
        if no_escolhido is not None:
            return no_escolhido, no_ref, custo_final
        else:
            return None
    
    # Adiciona o novo nó no trajeto, na posição que resulta o menor custo
    def incluir_novo_no(trajeto, cij, no_candidato):
        novo_trajeto = []
        if len(trajeto) == 1:
            novo_trajeto = [no_inicial, no_candidato, no_inicial]
        else:
            custo_final = float('inf')
            posicao_insercao = None
            for i in range(1, len(trajeto)):
                no_atual = trajeto[i]
                no_anterior = trajeto[i-1]
                custo_atual = cij[no_anterior][no_atual]
                novo_custo = cij[no_anterior][no_candidato] + cij[no_candidato][no_atual]
                custo_atualizado = novo_custo - custo_atual
                if custo_atualizado < custo_final:
                    custo_final = custo_atualizado
                    posicao_insercao = i
            novo_trajeto = trajeto[0:posicao_insercao] + [no_candidato] + trajeto[posicao_insercao:len(trajeto)] 
        return novo_trajeto
    
    # Executa as iterações:
    while len(trajeto) <= n:
        no_candidato, no_referencia, custo = escolher_no(trajeto, cij, no_inicial, mais_distante)
        trajeto = incluir_novo_no(trajeto, cij, no_candidato)
        
    # Cálcula o custo final do trajeto:
    custo_final = obter_custo_trajeto(trajeto, cij)
    
    # Adiciona 1 em cada valor de nós (para que os índices fiquem de 1 a n)
    trajeto_final = [no+1 for no in trajeto]
    
    return trajeto_final, custo_final

In [11]:
def resolver_heuristica_insercao_economica(cij, no_inicial=1):
    n = len(cij)
    no_inicial -= 1
    trajeto = [no_inicial, no_inicial]
    
    # Insere no trajeto o nó que resulta na rota mais econômica
    for k in range(n-1):
        custo_final = float('inf')
        posicao_insercao = None
        no_insercao = None
        for i in range(n):
            no_candidato = i
            if no_candidato not in trajeto:
                for j in range(1, len(trajeto)):
                    no_atual = trajeto[j]
                    no_anterior = trajeto[j-1]
                    custo_atual = cij[no_anterior][no_atual]
                    novo_custo = cij[no_anterior][no_candidato] + cij[no_candidato][no_atual]
                    custo_atualizado = novo_custo - custo_atual
                    if custo_atualizado < custo_final:
                        custo_final = custo_atualizado
                        posicao_insercao = j
                        no_insercao = no_candidato
        trajeto = trajeto[0:posicao_insercao] + [no_insercao] + trajeto[posicao_insercao:len(trajeto)]
    
    # Cálcula o custo final do trajeto:
    custo_final = obter_custo_trajeto(trajeto, cij)
    
    # Adiciona 1 em cada valor de nós (para que os índices fiquem de 1 a n)
    novo_trajeto = [no+1 for no in trajeto]
    
    return novo_trajeto, custo_final

Matriz de custos:

In [12]:
cij = [[0,    49,   46.6, 80.8],
       [49,   0,    59.8, 37],
       [46.6, 59.8, 0,    70.2],
       [80.8, 37,   70.2, 0]]

###  1 - Modelo com restrições de sub-rota de Dantzig

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

<br>

<b>Formulação:</b>

$\text{min }z = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$\sum\limits_{i \in Q} \sum\limits_{j \notin Q} x_{ij} \geq 1 \;\;\; \forall \; Q \subseteq \{1,...,n\}, 1 \leq |Q| \leq n-1$

$x_{ij} \in \{0,1\}$


#### 1.1 - Resolução

In [14]:
# Preparação do modelo para resolução via solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_preparacao_D = []
n_exec = 0

while n_exec < 100:
    start_time = time.time()

    # Lógica para criação dos conjuntos que irão originar as restrições de sub-rota de Dantzig
    Q, nQ = obter_conjuntos_dantzig(cij)
           
    # Declaração do modelo de designação:
    modelo_D = declarar_modelo_designacao(cij)

    # Inclusão das restrições de sub-rotas de Dantzig:
    modelo_D.restr_dantzig = pyo.ConstraintList()
    for q, nq in zip(Q, nQ):
        modelo_D.restr_dantzig.add(sum(modelo_D.x[i,j] for i in q for j in nq) >= 1)
    
    n_exec +=1
    tempo_preparacao_D.append(time.time() - start_time)

# Estatísticas da preparação do modelo:
obter_estatisticas(tempo_preparacao_D)

 Tempo médio: 0.0013 
 Desvio padrão: 0.0005


In [15]:
# Resolução através do solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_resolucao_D, resultado_D = resolver_modelo(modelo_D, n_exec=100)
print(resultado_D)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 13
  Number of nonzeros: 49
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04089045524597168
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [16]:
trajeto_D = obter_trajeto(modelo_D, cij)

[(1, 2), (2, 4), (4, 3), (3, 1)]


In [17]:
# Estatísticas da resolução do modelo:
obter_estatisticas(tempo_resolucao_D)

 Tempo médio: 0.0425 
 Desvio padrão: 0.0025


###  2 - Modelo com restrições de sub-rota MTZ

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

$u_i = \text{Variável auxiliar para definição das restrições de sub-rota MTZ} \;\;\; \forall \; i=1,...,m$

<br>

<b>Formulação:</b>

$\text{min }z = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$u_1 = 1$

$u_i - u_j + nx_{ij} \leq n-1 \;\;\; \forall \; i,j \in \{2,...,n\}, i \neq j$

$x_{ij} \in \{0,1\}$

$u_i \geq 0$

#### 2.1 - Resolução

In [18]:
# Preparação do modelo para resolução via solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final
n = len(cij)

tempo_preparacao_MTZ = []
n_exec = 0

while n_exec < 100:
    start_time = time.time()

   # Declaração do modelo:
    modelo_MTZ = declarar_modelo_designacao(cij)

    # Índice para a variável auxiliar u:
    modelo_MTZ.U = pyo.RangeSet(2,n)

    # Váriavel auxiliar u:
    modelo_MTZ.u = pyo.Var(modelo_MTZ.N, within=pyo.NonNegativeIntegers,bounds=(0,n-1))

    # Restrições de sub-rota MTZ:
    def f_MTZ(modelo, i, j):
        if i!=j: 
            return modelo.u[i] - modelo.u[j] + modelo.x[i,j] * n <= n-1
        else:
            return modelo.u[i] - modelo.u[i] == 0 # sem efeito no modelo

    modelo_MTZ.restr_MTZ = pyo.Constraint(modelo_MTZ.U, modelo_MTZ.N, rule=f_MTZ)
    
    n_exec +=1
    tempo_preparacao_MTZ.append(time.time() - start_time)

# Estatísticas da preparação do modelo:
obter_estatisticas(tempo_preparacao_MTZ)

 Tempo médio: 0.0013 
 Desvio padrão: 0.0005


In [19]:
# Resolução através do solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_MTZ, resultado_MTZ = resolver_modelo(modelo_MTZ, n_exec=100)
print(resultado_MTZ)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 21
  Number of variables: 17
  Number of nonzeros: 52
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.04188704490661621
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [20]:
trajeto_MTZ = obter_trajeto(modelo_MTZ, cij)

[(1, 2), (2, 4), (4, 3), (3, 1)]


In [21]:
obter_estatisticas(tempo_MTZ)

 Tempo médio: 0.0425 
 Desvio padrão: 0.002


###  3 - Modelo através da variante do branch and bound

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

<br>

<b>Formulação:</b>

$\text{min }z = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$x_{ij} \in \{0,1\}$

#### 3.1 - Resolução do modelo de designação

In [22]:
# Preparação do modelo para resolução via solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_preparacao_B = []
n_exec = 0

while n_exec < 100:
    start_time = time.time()

    # Declaração do modelo de designação:
    modelo_B = declarar_modelo_designacao(cij)

    n_exec +=1
    tempo_preparacao_B.append(time.time() - start_time)

# Estatísticas da preparação do modelo:
obter_estatisticas(tempo_preparacao_B)

 Tempo médio: 0.0009 
 Desvio padrão: 0.0004


In [23]:
# Resolução através do solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_resolucao_B, resultado_B = resolver_modelo(modelo_B, n_exec=100)
print(resultado_B)


Problem: 
- Name: unknown
  Lower bound: 167.2
  Upper bound: 167.2
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04089093208312988
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [24]:
trajeto_B = obter_trajeto(modelo_B, cij)
print("")
subrotas_B = obter_subrotas(trajeto_B)
print("")
custo_subrotas_B = obter_custo_subrotas(subrotas_B)

[(1, 3), (3, 1), (2, 4), (4, 2)]

Há 2 subrotas!
[[(1, 3), (3, 1)], [(2, 4), (4, 2)]]

{0: {'subrota': [(1, 3), (3, 1)], 'custo': 93.2}, 1: {'subrota': [(2, 4), (4, 2)], 'custo': 74}}


Necessário resolver os seguintes problemas:

- <b>Problema 00:</b>

    -$c_{13}=9999$
        
<br>
    
- <b>Problema 01:</b>

    -$c_{31}=9999$
    
<br>
    
- <b>Problema 02:</b>

    -$c_{24}=9999$
    
    
<br>
    
- <b>Problema 03:</b>

    -$c_{42}=9999$

#### 3.2 - Método branch and bound

##### 3.2.1 - Problema 00

In [25]:
# Preparação do modelo para resolução via solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_preparacao_B_00 = []
n_exec = 0

while n_exec < 100:
    start_time = time.time()

    # Atualização da matriz de custos:
    cij_00 = copy.deepcopy(cij)
    cij_00[0][2] = 999999 # [1,3]

    # Declaração do modelo:
    modelo_B_00 = declarar_modelo_designacao(cij_00)

    n_exec +=1
    tempo_preparacao_B_00.append(time.time() - start_time)

# Estatísticas da preparação do modelo:
obter_estatisticas(tempo_preparacao_B_00)

 Tempo médio: 0.001 
 Desvio padrão: 0.0004


In [26]:
# Matriz de custos utilizada:
cij_00

[[0, 49, 999999, 80.8],
 [49, 0, 59.8, 37],
 [46.6, 59.8, 0, 70.2],
 [80.8, 37, 70.2, 0]]

In [27]:
# Resolução através do solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_resolucao_B_00, resultado_B_00 = resolver_modelo(modelo_B_00, n_exec=100)
print(resultado_B_00)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04089021682739258
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [28]:
trajeto_B_00 = obter_trajeto(modelo_B_00, cij)
print("")
subrotas_B_00 = obter_subrotas(trajeto_B_00)
print("")
custo_subrotas_B_00 = obter_custo_subrotas(subrotas_B_00)

[(1, 2), (2, 4), (4, 3), (3, 1)]

Não há subrotas!
[[(1, 2), (2, 4), (4, 3), (3, 1)]]

{0: {'subrota': [(1, 2), (2, 4), (4, 3), (3, 1)], 'custo': 202.8}}


##### 3.2.2 - Problema 01

In [29]:
tempo_preparacao_B_01 = []
n_exec = 0

while n_exec < 100:
    start_time = time.time()

    # Atualização da matriz de custos:
    cij_01 = copy.deepcopy(cij)
    cij_01[2][0] = 999999 # [3,1]

    # Declaração do modelo:
    modelo_B_01 = declarar_modelo_designacao(cij_01)

    n_exec +=1
    tempo_preparacao_B_01.append(time.time() - start_time)

# Estatísticas da preparação do modelo:
obter_estatisticas(tempo_preparacao_B_01)

 Tempo médio: 0.001 
 Desvio padrão: 0.0007


In [30]:
# Matriz de custos utilizada:
cij_01

[[0, 49, 46.6, 80.8],
 [49, 0, 59.8, 37],
 [999999, 59.8, 0, 70.2],
 [80.8, 37, 70.2, 0]]

In [31]:
# Resolução através do solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_resolucao_B_01, resultado_B_01 = resolver_modelo(modelo_B_01, n_exec=100)
print(resultado_B_01)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04288506507873535
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [36]:
trajeto_B_01 = obter_trajeto(modelo_B_01, cij)
print("")
subrotas_B_01 = obter_subrotas(trajeto_B_01)
print("")
custo_subrotas_B_01 = obter_custo_subrotas(subrotas_B_01)

[(1, 3), (3, 4), (4, 2), (2, 1)]

Não há subrotas!
[[(1, 3), (3, 4), (4, 2), (2, 1)]]

{0: {'subrota': [(1, 3), (3, 4), (4, 2), (2, 1)], 'custo': 202.8}}


##### 3.2.3 - Problema 02

In [33]:
tempo_preparacao_B_02 = []
n_exec = 0

while n_exec < 100:
    start_time = time.time()

    # Atualização da matriz de custos:
    cij_02 = copy.deepcopy(cij)
    cij_02[1][3] = 999999 # [2,4]

    # Declaração do modelo:
    modelo_B_02 = declarar_modelo_designacao(cij_02)

    n_exec +=1
    tempo_preparacao_B_02.append(time.time() - start_time)

# Estatísticas da preparação do modelo:
obter_estatisticas(tempo_preparacao_B_02)

 Tempo médio: 0.0009 
 Desvio padrão: 0.0004


In [34]:
# Matriz de custos utilizada:
cij_02

[[0, 49, 46.6, 80.8],
 [49, 0, 59.8, 999999],
 [46.6, 59.8, 0, 70.2],
 [80.8, 37, 70.2, 0]]

In [35]:
# Resolução através do solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_resolucao_B_02, resultado_B_02 = resolver_modelo(modelo_B_02, n_exec=100)
print(resultado_B_02)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04188871383666992
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [37]:
trajeto_B_02 = obter_trajeto(modelo_B_02, cij)
print("")
subrotas_B_02 = obter_subrotas(trajeto_B_02)
print("")
custo_subrotas_B_02 = obter_custo_subrotas(subrotas_B_02)

[(1, 3), (3, 4), (4, 2), (2, 1)]

Não há subrotas!
[[(1, 3), (3, 4), (4, 2), (2, 1)]]

{0: {'subrota': [(1, 3), (3, 4), (4, 2), (2, 1)], 'custo': 202.8}}


##### 3.2.4 - Problema 03

In [39]:
tempo_preparacao_B_03 = []
n_exec = 0

while n_exec < 100:
    start_time = time.time()

    # Atualização da matriz de custos:
    cij_03 = copy.deepcopy(cij)
    cij_03[3][1] = 999999 # [4,2]

    # Declaração do modelo:
    modelo_B_03 = declarar_modelo_designacao(cij_03)

    n_exec +=1
    tempo_preparacao_B_03.append(time.time() - start_time)

# Estatísticas da preparação do modelo:
obter_estatisticas(tempo_preparacao_B_03)

 Tempo médio: 0.001 
 Desvio padrão: 0.0005


In [40]:
# Matriz de custos utilizada:
cij_03

[[0, 49, 46.6, 80.8],
 [49, 0, 59.8, 37],
 [46.6, 59.8, 0, 70.2],
 [80.8, 999999, 70.2, 0]]

In [41]:
# Resolução através do solver GLPK
# Calcula o tempo médio de 100 execuções para o relatório final

tempo_resolucao_B_03, resultado_B_03 = resolver_modelo(modelo_B_03, n_exec=100)
print(resultado_B_03)


Problem: 
- Name: unknown
  Lower bound: 202.8
  Upper bound: 202.8
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04088950157165527
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



###  3 - Heurística de inserção do mais próximo

In [ ]:
# Executa 10 vezes para obter o tempo médio:

t_himp = []
n_exec = 0
while n_exec < 10:
    start_time = time.time()
    trajeto_himp, custo_himp = resolver_heuristica_insercao(cij)
    n_exec +=1
    t_himp.append(time.time() - start_time)

obter_estatisticas(t_himp)

In [ ]:
print("Trajeto final:", trajeto_himp)
print("Custo final:", custo_himp)

###  4 - Heurística de inserção do mais distante

In [ ]:
# Executa 10 vezes para obter o tempo médio:

t_himd = []
n_exec = 0
while n_exec < 10:
    start_time = time.time()
    trajeto_himd, custo_himd = resolver_heuristica_insercao(cij, mais_distante=True)
    n_exec +=1
    t_himd.append(time.time() - start_time)

obter_estatisticas(t_himd)

In [ ]:
print("Trajeto final:", trajeto_himd)
print("Custo final:", custo_himd)

###  5 - Heurística de inserção mais econômica

In [ ]:
# Executa 10 vezes para obter o tempo médio:

t_hime = []
n_exec = 0
while n_exec < 10:
    start_time = time.time()
    trajeto_hime, custo_hime = resolver_heuristica_insercao_economica(cij)
    n_exec +=1
    t_hime.append(time.time() - start_time)

obter_estatisticas(t_hime)

In [ ]:
print("Trajeto final:", trajeto_hime)
print("Custo final:", custo_hime)